# 1.0 ‎ **Installation & Setup**

In this notebook, we prepare the datasets required for time series forecasting. \
We install essential packages and set up paths for accessing weather, socioeconomic, geospatial, and municipal issue datasets.


In [ ]:
!pip install --upgrade --quiet pandas numpy flashtext rapidfuzz openai holidays

In [78]:
# Standard Libraries
import pandas as pd
import numpy as np
import json
import os
import re

# Libraries for Data Engineering
from flashtext import KeywordProcessor
from rapidfuzz import fuzz
from openai import OpenAI
import holidays

# Display full column width
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)

# 1.1 **Preparing the Data**

### 1.1.1 **Define Dataset Paths**

In [79]:
# Define paths to raw datasets
DATA_DIR = "../data/datasets"

# Define countries and their respective cities
regions = {
    "us": ["newyork"],
    # "us": ["chicago", "sanfrancisco", "newyork"],
    # "uk": ["london", "manchester"],             
    # "sg": ["singapore"]
}

# Store all paths in a nested dictionary: datasets[country][city][category]
datasets = {}
for country, cities in regions.items():
    datasets[country] = {}
    for city in cities:
        city_key = city.lower()
        datasets[country][city_key] = {
            "municipal": os.path.join(DATA_DIR, f"municipal_reports/{country}", f"municipal_{country}_{city_key}_311.csv"),
            "weather": os.path.join(DATA_DIR, f"weather/{country}", f"weather_{country}_{city_key}_open-meteo.csv"),
            "geospatial": os.path.join(DATA_DIR, f"geospatial/{country}", f"geospatial_{country}_{city_key}.csv"),
            "socioeconomic": os.path.join(DATA_DIR, f"socioeconomic/{country}", f"socioeconomic_{country}_{city_key}.csv"),
            "boundaries": os.path.join(DATA_DIR, f"boundaries/{country}", f"boundaries_{country}_{city_key}.csv"),
            "clusters": os.path.join(DATA_DIR, f"clusters/{country}", f"clusters_{country}_{city_key}.csv"),
        }

### 1.1.2 **Load Datasets**

In [91]:
city_dfs = {}

for country in datasets:
    city_dfs[country] = {}
    for city in datasets[country]:
        city_dfs[country][city] = {}
        for category in datasets[country][city]:
            path = datasets[country][city][category].replace("\\", "/")
            if os.path.exists(path):
                print(f"Loading {path}")
                city_dfs[country][city][category] = pd.read_csv(path, low_memory=False)
            else:
                print(f"File not found: {path}")
                city_dfs[country][city][category] = None

Loading ../data/datasets/municipal_reports/us/municipal_us_newyork_311.csv
Loading ../data/datasets/weather/us/weather_us_newyork_open-meteo.csv
Loading ../data/datasets/geospatial/us/geospatial_us_newyork.csv
Loading ../data/datasets/socioeconomic/us/socioeconomic_us_newyork.csv
Loading ../data/datasets/boundaries/us/boundaries_us_newyork.csv
Loading ../data/datasets/clusters/us/clusters_us_newyork.csv


# 1.2 **Understanding the Data**

### 1.2.1 **Municipal Issues Dataset**

**Basic Structure and Dimensions**

In [81]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("municipal")
        if df is not None:
            print(f"{country.upper()} | {city.capitalize()} Municipal Shape: {df.shape}")
        else:
            print(f"{country.upper()} | {city.capitalize()} Municipal: Data not loaded or missing")


US | Newyork Municipal Shape: (881879, 41)


**Columns, Data Types and Non-Null Overview**

In [82]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("municipal")
        print(f"\n{country.upper()} | {city.capitalize()} Municipal Info:")
        if df is not None:
            df.info()
        else:
            print("Data not loaded or missing.")


US | Newyork Municipal Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881879 entries, 0 to 881878
Data columns (total 41 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   unique_key                      881879 non-null  int64  
 1   created_date                    881879 non-null  object 
 2   closed_date                     826396 non-null  object 
 3   agency                          881879 non-null  object 
 4   agency_name                     881879 non-null  object 
 5   complaint_type                  881879 non-null  object 
 6   descriptor                      865096 non-null  object 
 7   location_type                   786899 non-null  object 
 8   incident_zip                    878971 non-null  float64
 9   incident_address                853217 non-null  object 
 10  street_name                     853198 non-null  object 
 11  cross_street_1                  575509 non-null 

**Summary of Key Stats**

In [41]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("municipal")
        if df is not None:
            print(f"\n{country.upper()} | {city.capitalize()} Municipal Describe:")
            print(df.describe(include='all'))
        else:
            print(f"\n{country.upper()} | {city.capitalize()} Municipal: Data not loaded or missing")


US | Newyork Municipal Describe:
          unique_key             created_date              closed_date  agency                      agency_name   complaint_type        descriptor  \
count   8.818790e+05                   881879                   826396  881879                           881879           881879            865096   
unique           NaN                   729284                   540597      15                               15              171               803   
top              NaN  2025-03-03T07:00:00.000  2025-03-04T00:00:00.000    NYPD  New York City Police Department  Illegal Parking  Loud Music/Party   
freq             NaN                       31                      409  389877                           389877           140106            107373   
mean    6.405509e+07                      NaN                      NaN     NaN                              NaN              NaN               NaN   
std     2.741836e+05                      NaN                     

**Issue Type Diversity**

In [111]:
# Map (country, city) to relevant column for issue type
issue_type_columns = {
    ("us", "chicago"): "sr_type",
    ("us", "sanfrancisco"): "service_name",
    ("us", "newyork"): "complaint_type",
}

for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("municipal")
        column = issue_type_columns.get((country, city))
        
        if df is not None and column in df.columns:
            print(f"{country.upper()} | {city.capitalize()} Issue Types ({column}):")
            print(df[column].value_counts().head(50), "\n")
        elif df is not None:
            print(f"{country.upper()} | {city.capitalize()}: Column '{column}' not found in DataFrame.\n")
        else:
            print(f"{country.upper()} | {city.capitalize()}: Municipal DataFrame not loaded.\n")


US | Newyork Issue Types (complaint_type):
complaint_type
Illegal Parking                   140106
HEAT/HOT WATER                    134939
Noise - Residential               127225
Blocked Driveway                   41726
UNSANITARY CONDITION               26249
PLUMBING                           19018
Abandoned Vehicle                  16691
Noise - Street/Sidewalk            16111
Water System                       15815
PAINT/PLASTER                      14901
Noise - Commercial                 13989
Traffic Signal Condition           13818
Street Condition                   13451
Noise                              13121
Dirty Condition                    12654
DOOR/WINDOW                        12195
Derelict Vehicles                   9945
WATER LEAK                          9721
Missed Collection                   8831
Illegal Dumping                     8426
GENERAL                             8224
Snow or Ice                         7873
ELECTRIC                            7867

**Date Range Checks**

In [43]:
# Define city-specific date columns
date_columns = {
    ("us", "chicago"): "created_date",
    ("us", "sanfrancisco"): "requested_datetime",
    ("us", "newyork"): "created_date",
}

# Loop through city DataFrames and print date ranges
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("municipal")
        if df is not None:
            date_col = date_columns.get((country, city))
            
            if date_col in df.columns:
                try:
                    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                    min_date = df[date_col].min()
                    max_date = df[date_col].max()
                    print(f"{country.upper()} | {city.capitalize()} Date Range: {min_date.date()} to {max_date.date()}")
                except Exception as e:
                    print(f"{country.upper()} | {city.capitalize()} Date Range: Error parsing dates – {str(e)}")
            else:
                print(f"{country.upper()} | {city.capitalize()}: Date column '{date_col}' not found in DataFrame")
        else:
            print(f"{country.upper()} | {city.capitalize()} Municipal: Data not loaded or missing")


US | Newyork Date Range: 2025-01-01 to 2025-04-01


### 1.2.2 **Weather Dataset**

**Basic Structure and Dimensions**

In [92]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("weather")
        if df is not None:
            print(f"{country.upper()} | {city.capitalize()} Weather Shape: {df.shape}")
        else:
            print(f"{country.upper()} | {city.capitalize()} Weather: Data not loaded or missing")

US | Newyork Weather Shape: (226619, 15)


**Columns, Data Types and Non-Null Overview**

In [93]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("weather")
        print(f"\n{country.upper()} | {city.capitalize()} Weather Info:")
        if df is not None:
            df.info()
        else:
            print("Data not loaded or missing.")


US | Newyork Weather Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226619 entries, 0 to 226618
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   report_id             226619 non-null  int64  
 1   latitude              226619 non-null  float64
 2   longitude             226619 non-null  float64
 3   reported_datetime     226619 non-null  object 
 4   weather_datetime      226619 non-null  object 
 5   temperature_2m        226619 non-null  float64
 6   relative_humidity_2m  226619 non-null  int64  
 7   precipitation         226619 non-null  float64
 8   windspeed_10m         226619 non-null  float64
 9   pm10                  226619 non-null  float64
 10  pm2_5                 226619 non-null  float64
 11  carbon_monoxide       226619 non-null  int64  
 12  nitrogen_dioxide      226619 non-null  float64
 13  ozone                 226619 non-null  int64  
 14  sulphur_dioxide       22

**Viewing the Data**

In [95]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("weather")
        print(f"{country.upper()} | {city.capitalize()} Weather First 3 Rows:")
        if df is not None:
            print(df.head(3))
        else:
            print("Data not loaded or missing.")
        print()

US | Newyork Weather First 3 Rows:
   report_id   latitude  longitude        reported_datetime weather_datetime  temperature_2m  relative_humidity_2m  precipitation  windspeed_10m  \
0   64526404  40.844482  -73.89044  2025-04-01T23:47:26.000    1/4/2025 2:00            14.4                    98            3.7            8.6   
1   64526404  40.844482  -73.89044  2025-04-01T23:47:26.000    1/4/2025 3:00            14.2                    98            4.0            9.3   
2   64526404  40.844482  -73.89044  2025-04-01T23:47:26.000    1/4/2025 4:00            14.9                    97            2.4            9.4   

   pm10  pm2_5  carbon_monoxide  nitrogen_dioxide  ozone  sulphur_dioxide  
0  17.3   16.9              390              72.4     11              6.4  
1  20.1   19.9              375              71.3     10              6.6  
2  22.5   22.3              330              58.6     20              6.2  



**Summary of Key Stats**

In [99]:
columns_to_describe = ["temperature_2m", "relative_humidity_2m", "precipitation", "windspeed_10m", "pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "ozone", "sulphur_dioxide"]
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("weather")
        if df is not None:
            cols = [col for col in columns_to_describe if col in df.columns]
            if cols:
                print(f"\n{country.upper()} | {city.capitalize()} Weather Describe (Selected Columns):")
                print(df[cols].describe(include='all'))
            else:
                print(f"\n{country.upper()} | {city.capitalize()} Weather: None of the selected columns found")
        else:
            print(f"\n{country.upper()} | {city.capitalize()} Weather: Data not loaded or missing")


US | Newyork Weather Describe (Selected Columns):
       temperature_2m  relative_humidity_2m  precipitation  windspeed_10m           pm10          pm2_5  carbon_monoxide  nitrogen_dioxide  \
count   226619.000000         226619.000000  226619.000000  226619.000000  226619.000000  226619.000000    226619.000000     226619.000000   
mean        13.286737             82.158654       1.104199      13.668010      11.141519      10.558537       243.054576         27.900215   
std          4.770037             16.001017       1.749093       5.562395       5.887698       5.689661        57.197623         20.864907   
min          3.400000             36.000000       0.000000       1.300000       0.800000       0.800000       145.000000          2.400000   
25%          9.100000             73.000000       0.000000       8.800000       7.500000       6.700000       211.000000          9.700000   
50%         14.200000             87.000000       0.100000      13.800000      10.900000      10.

### 1.2.3 **Geospatial Dataset**

**Basic Structure and Dimensions**

In [96]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("geospatial")
        if df is not None:
            print(f"{country.upper()} | {city.capitalize()} Geospatial Shape: {df.shape}")
        else:
            print(f"{country.upper()} | {city.capitalize()} Geospatial: Data not loaded or missing")

US | Newyork Geospatial Shape: (881879, 13)


**Columns, Data Types and Non-Null Overview**

In [97]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("geospatial")
        print(f"\n{country.upper()} | {city.capitalize()} Geospatial Info:")
        if df is not None:
            df.info()
        else:
            print("Data not loaded or missing.")


US | Newyork Geospatial Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881879 entries, 0 to 881878
Data columns (total 13 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   report_id                         881879 non-null  int64  
 1   latitude                          881879 non-null  float64
 2   longitude                         881879 non-null  float64
 3   commercial_count_within_200m      881879 non-null  int64  
 4   dist_to_nearest_commercial_in_m   881879 non-null  float64
 5   residential_count_within_200m     881879 non-null  int64  
 6   dist_to_nearest_residential_in_m  881879 non-null  float64
 7   facilities_count_within_200m      881879 non-null  int64  
 8   dist_to_nearest_facilities_in_m   881879 non-null  float64
 9   recreation_count_within_200m      881879 non-null  int64  
 10  dist_to_nearest_recreation_in_m   881879 non-null  float64
 11  transit_count_within_

**Viewing the Data**

In [98]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("geospatial")
        print(f"{country.upper()} | {city.capitalize()} Geospatial First 3 Rows:")
        if df is not None:
            print(df.head(3))
        else:
            print("Data not loaded or missing.")
        print()

US | Newyork Geospatial First 3 Rows:
   report_id   latitude  longitude  commercial_count_within_200m  dist_to_nearest_commercial_in_m  residential_count_within_200m  \
0   64526847  40.592989 -73.784356                             1                        58.941575                              1   
1   64525572  40.691837 -73.926142                            11                        17.396196                             21   
2   64525039  40.814843 -73.900012                             0                       360.861254                              0   

   dist_to_nearest_residential_in_m  facilities_count_within_200m  dist_to_nearest_facilities_in_m  recreation_count_within_200m  \
0                        149.001010                             0                       325.290183                             1   
1                         31.604263                             0                       354.273924                             3   
2                        679.911540  

**Summary of Key Stats**

In [100]:
columns_to_describe = ["commercial_count_within_200m", "dist_to_nearest_commercial_in_m", "residential_count_within_200m", "dist_to_nearest_residential_in_m", "facilities_count_within_200m", "dist_to_nearest_facilities_in_m", "recreation_count_within_200m", "dist_to_nearest_recreation_in_m", "transit_count_within_200m", "dist_to_nearest_transit_in_m"]
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("geospatial")
        if df is not None:
            cols = [col for col in columns_to_describe if col in df.columns]
            if cols:
                print(f"\n{country.upper()} | {city.capitalize()} Geospatial Describe (Selected Columns):")
                print(df[cols].describe(include='all'))
            else:
                print(f"\n{country.upper()} | {city.capitalize()} Geospatial: None of the selected columns found")
        else:
            print(f"\n{country.upper()} | {city.capitalize()} Geospatial: Data not loaded or missing")



US | Newyork Geospatial Describe (Selected Columns):
       commercial_count_within_200m  dist_to_nearest_commercial_in_m  residential_count_within_200m  dist_to_nearest_residential_in_m  \
count                 881879.000000                    881879.000000                  881879.000000                     881879.000000   
mean                       5.319627                       227.489792                       5.726629                        360.550419   
std                        9.627804                       219.401600                      15.824878                        383.847987   
min                        0.000000                         1.360689                       0.000000                          0.907247   
25%                        0.000000                        76.374871                       0.000000                         79.184939   
50%                        1.000000                       173.496892                       0.000000                        2

### 1.2.4 **Socioeconomic Dataset**

**Basic Structure and Dimensions**

In [101]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("socioeconomic")
        if df is not None:
            print(f"{country.upper()} | {city.capitalize()} Socioeconomic Shape: {df.shape}")
        else:
            print(f"{country.upper()} | {city.capitalize()} Socioeconomic: Data not loaded or missing")

US | Newyork Socioeconomic Shape: (881846, 7)


**Columns, Data Types and Non-Null Overview**

In [102]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("socioeconomic")
        print(f"\n{country.upper()} | {city.capitalize()} Socioeconomic Info:")
        if df is not None:
            df.info()
        else:
            print("Data not loaded or missing.")


US | Newyork Socioeconomic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881846 entries, 0 to 881845
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   report_id            881846 non-null  int64  
 1   reported_datetime    881846 non-null  object 
 2   GEOID                881846 non-null  int64  
 3   reported_datetime.1  881846 non-null  object 
 4   median_income        869392 non-null  float64
 5   total_population     881846 non-null  float64
 6   average_age          875311 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 47.1+ MB


**Viewing the Data**

In [103]:
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("socioeconomic")
        print(f"{country.upper()} | {city.capitalize()} Socioeconomic First 3 Rows:")
        if df is not None:
            print(df.head(3))
        else:
            print("Data not loaded or missing.")
        print()

US | Newyork Socioeconomic First 3 Rows:
   report_id        reported_datetime        GEOID      reported_datetime.1  median_income  total_population  average_age
0   64526847  2025-04-01T23:59:26.000  36081097204  2025-04-01T23:59:26.000        73207.0            4218.0         43.7
1   64525572  2025-04-01T23:58:49.000  36047039500  2025-04-01T23:58:49.000        52188.0            3676.0         31.5
2   64525039  2025-04-01T23:58:38.000  36005008300  2025-04-01T23:58:38.000        50920.0            6564.0         34.0



**Summary of Key Stats**

In [104]:
columns_to_describe = ["median_income", "total_population", "average_age"]
for country in city_dfs:
    for city in city_dfs[country]:
        df = city_dfs[country][city].get("socioeconomic")
        if df is not None:
            cols = [col for col in columns_to_describe if col in df.columns]
            if cols:
                print(f"\n{country.upper()} | {city.capitalize()} Socioeconomic Describe (Selected Columns):")
                print(df[cols].describe(include='all'))
            else:
                print(f"\n{country.upper()} | {city.capitalize()} Socioeconomic: None of the selected columns found")
        else:
            print(f"\n{country.upper()} | {city.capitalize()} Socioeconomic: Data not loaded or missing")


US | Newyork Socioeconomic Describe (Selected Columns):
       median_income  total_population    average_age
count  869392.000000     881846.000000  875311.000000
mean    78785.041608       4651.341462      37.631507
std     43335.877511       2199.175880       6.171763
min     11406.000000          0.000000      12.600000
25%     46645.000000       3171.000000      33.900000
50%     71250.000000       4396.000000      36.700000
75%     98269.000000       5774.000000      40.700000
max    250001.000000      15945.000000      82.900000


# 1.3 **Cleaning the Data**

### 1.3.1 **Aggregating the Weather Datasets**

In [ ]:
def aggregate_weather_df(df_weather):
    if df_weather is None:
        return None

    # Drop weather_datetime
    df_weather = df_weather.drop(columns=["weather_datetime"], errors="ignore")

    # Get first issue_datetime per id
    issue_times = df_weather.groupby("report_id", as_index=False)["reported_datetime"].first()

    # Average other numeric columns
    df_avg = df_weather.drop(columns=["reported_datetime"]).groupby("report_id", as_index=False).mean(numeric_only=True)

    # Merge back issue_datetime
    df_aggregated = pd.merge(issue_times, df_avg, on="report_id")
    
    return df_aggregated

In [106]:
for country in city_dfs:
    for city in city_dfs[country]:
        df_weather = city_dfs[country][city].get("weather")
        if df_weather is not None:
            df_weather_agg = aggregate_weather_df(df_weather)
            city_dfs[country][city]["weather_agg"] = df_weather_agg
            print(f"{country.upper()} - {city.capitalize()}: Weather aggregated with shape {df_weather_agg.shape}. Access it via: city_dfs['{country}']['{city}']['weather_agg'].")
        else:
            print(f"{country.upper()} - {city.capitalize()}: Weather data missing or not loaded.")

KeyError: 'Column not found: issue_datetime'

### 1.3.2 **Schema Unification**

In [168]:
column_mapping = {
    "chicago": {
        "sr_number": "id",
        "created_date": "reported_datetime",
        "closed_date": "closed_datetime",
        "status": "status",
        "latitude": "latitude",
        "longitude": "longitude",
        "sr_type": "issue_type"
    },
    "sanfrancisco": {
        "service_request_id": "id",
        "requested_datetime": "reported_datetime",
        "closed_date": "closed_datetime",
        "status_notes": "status",
        "lat": "latitude",
        "long": "longitude",
        "service_name": "issue_type"
    },
    "newyork": {
        "unique_key": "id",
        "created_date": "reported_datetime",
        "closed_date": "closed_datetime",
        "status": "status",
        "latitude": "latitude",
        "longitude": "longitude",
        "complaint_type": "issue_type"
    }
}

In [169]:
for country in city_dfs:
    for city in city_dfs[country]:
        city_key = city.lower()
        df = city_dfs[country][city].get("municipal")

        if df is not None and city_key in column_mapping:
            renamed_df = df.rename(columns=column_mapping[city_key])
            city_dfs[country][city]["municipal_unified"] = renamed_df
            print(f"{country.upper()} - {city.capitalize()}: Municipal schema unified. Access it via: city_dfs['{country}']['{city}']['municipal_unified'].")
        else:
            print(f"{country.upper()} - {city.capitalize()}: No municipal data or no column mapping defined.")

US - Chicago: Municipal schema unified. Access it via: city_dfs['us']['chicago']['municipal_unified'].
US - Sanfrancisco: Municipal schema unified. Access it via: city_dfs['us']['sanfrancisco']['municipal_unified'].
US - Newyork: Municipal schema unified. Access it via: city_dfs['us']['newyork']['municipal_unified'].


### 1.3.3 **Remove Unnecessary Columns**

In [170]:
def filter_columns(df, keep_columns):
    """
    Keep only the specified columns from a DataFrame.

    Parameters:
    - df: Pandas DataFrame to filter
    - keep_columns: List of column names to retain

    Returns:
    - Filtered DataFrame
    """
    if df is None:
        return None

    # Keep only columns that exist in the DataFrame
    valid_columns = [col for col in keep_columns if col in df.columns]
    return df[valid_columns].copy()

In [174]:
columns_to_keep = {
    "municipal_unified": ["id", "status", "reported_datetime", "closed_datetime", "issue_type", "latitude", "longitude"],
    "weather_agg": ["id", "temp_c", "humidity", "wind_kph", "precip_mm", "pm10", "pm2_5", "co", "no2", "o3", "so2"],
    "geospatial": ["id", "commercial_count_within_200m", "dist_to_nearest_commericial", "recreation_count_within_200m", "dist_to_nearest_recreation", "facilities_count_within_200m", "dist_to_nearest_facility", "transit_count_within_200m", "dist_to_nearest_transit"],
    "socioeconomic": ["id", "boundary_name", "area_km2", "median_income", "total_population", "average_age"]
}

In [175]:
for country in city_dfs:
    for city in city_dfs[country]:
        for category, keep_cols in columns_to_keep.items():
            df = city_dfs[country][city].get(category)
            if df is not None:
                filtered_df = filter_columns(df, keep_cols)
                filtered_key = f"{category}_filtered"
                city_dfs[country][city][filtered_key] = filtered_df
                # print(f"{country.upper()} - {city.capitalize()} - {category}: Filtered to columns {filtered_df.columns.tolist()}. Access it via: city_dfs['{country}']['{city}']['{category}_filtered'].")
                print(f"{country.upper()} - {city.capitalize()} - {category}: Filtered to columns. Access it via: city_dfs['{country}']['{city}']['{category}_filtered'].")
            else:
                print(f"{country.upper()} - {city.capitalize()} - {category}: Not loaded or missing")

US - Chicago - municipal_unified: Filtered to columns. Access it via: city_dfs['us']['chicago']['municipal_unified_filtered'].
US - Chicago - weather_agg: Filtered to columns. Access it via: city_dfs['us']['chicago']['weather_agg_filtered'].
US - Chicago - geospatial: Filtered to columns. Access it via: city_dfs['us']['chicago']['geospatial_filtered'].
US - Chicago - socioeconomic: Filtered to columns. Access it via: city_dfs['us']['chicago']['socioeconomic_filtered'].
US - Sanfrancisco - municipal_unified: Filtered to columns. Access it via: city_dfs['us']['sanfrancisco']['municipal_unified_filtered'].
US - Sanfrancisco - weather_agg: Filtered to columns. Access it via: city_dfs['us']['sanfrancisco']['weather_agg_filtered'].
US - Sanfrancisco - geospatial: Filtered to columns. Access it via: city_dfs['us']['sanfrancisco']['geospatial_filtered'].
US - Sanfrancisco - socioeconomic: Filtered to columns. Access it via: city_dfs['us']['sanfrancisco']['socioeconomic_filtered'].
US - Newyork

### 1.3.4 **Remove Bad or Missing Values**

In [177]:
def clean_dataframe(df, additional_bad_values=None):
    """
    Remove rows with missing or bad values.

    Parameters:
    - df: Pandas DataFrame to clean
    - additional_bad_values: List of extra values to treat as missing (e.g., ["NA", "?", "-"])

    Returns:
    - Cleaned DataFrame
    """
    if df is None:
        return None

    # Replace known bad values with NA
    if additional_bad_values:
        df = df.replace(additional_bad_values, pd.NA)

    # Drop rows with any missing values
    df_cleaned = df.dropna(how="any").reset_index(drop=True)
    return df_cleaned

In [178]:
categories = ["municipal_unified_filtered", "weather_agg_filtered", "geospatial_filtered", "socioeconomic_filtered"]

for country in city_dfs:
    for city in city_dfs[country]:
        for category in categories:
            df = city_dfs[country][city].get(category)
            if df is not None:
                cleaned_df = clean_dataframe(df, additional_bad_values=["NA", "?", "-", "null", "N/A", ""])
                clean_key = f"{category}_clean"
                city_dfs[country][city][clean_key] = cleaned_df
                print(f"{country.upper()} - {city.capitalize()} - {category}: Cleaned shape {cleaned_df.shape}. Access it via: city_dfs['{country}']['{city}']['{category}_clean'].")
            else:
                print(f"{country.upper()} - {city.capitalize()} - {category}: Not loaded or missing")


US - Chicago - municipal_unified_filtered: Cleaned shape (80988, 7). Access it via: city_dfs['us']['chicago']['municipal_unified_filtered_clean'].
US - Chicago - weather_agg_filtered: Cleaned shape (3364, 11). Access it via: city_dfs['us']['chicago']['weather_agg_filtered_clean'].
US - Chicago - geospatial_filtered: Cleaned shape (99852, 9). Access it via: city_dfs['us']['chicago']['geospatial_filtered_clean'].
US - Chicago - socioeconomic_filtered: Cleaned shape (99852, 6). Access it via: city_dfs['us']['chicago']['socioeconomic_filtered_clean'].
US - Sanfrancisco - municipal_unified_filtered: Cleaned shape (87177, 7). Access it via: city_dfs['us']['sanfrancisco']['municipal_unified_filtered_clean'].
US - Sanfrancisco - weather_agg_filtered: Cleaned shape (3493, 11). Access it via: city_dfs['us']['sanfrancisco']['weather_agg_filtered_clean'].
US - Sanfrancisco - geospatial_filtered: Cleaned shape (98314, 9). Access it via: city_dfs['us']['sanfrancisco']['geospatial_filtered_clean'].
U

### 1.3.5 **Combining and Merging Datasets**

In [179]:
combined_by_category = {}
target_categories = ["municipal_unified_filtered_clean", "weather_agg_filtered_clean", "geospatial_filtered_clean", "socioeconomic_filtered_clean"]

for category in target_categories:
    base_name = category.split("_")[0]  # Get 'municipal', 'weather', etc.
    combined_key = f"{base_name}_combined"

    frames = []
    for country in city_dfs:
        for city in city_dfs[country]:
            df = city_dfs[country][city].get(category)
            if df is not None:
                df = df.copy()
                df["city"] = city
                df["country"] = country
                frames.append(df)

    if frames:
        combined_by_category[combined_key] = pd.concat(frames, ignore_index=True)
        print(f"{combined_key}: Combined shape = {combined_by_category[combined_key].shape}. Access it via: combined_by_category['{base_name}_combined'].")
    else:
        print(f"{combined_key}: No valid data found to combine.")

municipal_combined: Combined shape = (244192, 9). Access it via: combined_by_category['municipal_combined'].
weather_combined: Combined shape = (16659, 13). Access it via: combined_by_category['weather_combined'].
geospatial_combined: Combined shape = (296516, 11). Access it via: combined_by_category['geospatial_combined'].
socioeconomic_combined: Combined shape = (296489, 8). Access it via: combined_by_category['socioeconomic_combined'].


In [180]:
# Keep city and country only from municipal_combined
combined_by_category["weather_combined"] = combined_by_category["weather_combined"].drop(columns=["city", "country"], errors="ignore")
combined_by_category["geospatial_combined"] = combined_by_category["geospatial_combined"].drop(columns=["city", "country"], errors="ignore")
combined_by_category["socioeconomic_combined"] = combined_by_category["socioeconomic_combined"].drop(columns=["city", "country"], errors="ignore")

# Start with municipal
df_merged = combined_by_category.get("municipal_combined")

# Sequentially merge with the rest
for category in ["weather_combined", "geospatial_combined", "socioeconomic_combined"]:
    df_to_merge = combined_by_category.get(category)
    if df_to_merge is not None:
        df_merged = df_merged.merge(df_to_merge, on="id", how="inner", suffixes=('', f'_{category}'))
        print(f"Merged with {category}: Result shape = {df_merged.shape}")
    else:
        print(f"{category}: Skipped (not available)")

Merged with weather_combined: Result shape = (10115, 19)
Merged with geospatial_combined: Result shape = (10115, 27)
Merged with socioeconomic_combined: Result shape = (10115, 32)


### 1.3.6 **Datatype Correction**

In [184]:
def enforce_column_dtypes(df, dtype_mapping):
    """
    Convert specified columns to desired data types if not already correct.

    Parameters:
    - df: The pandas DataFrame to modify.
    - dtype_mapping: A dictionary in the form {"col_name": desired_dtype}

    Returns:
    - Modified DataFrame with columns converted as needed.
    """
    for col, desired_type in dtype_mapping.items():
        if col not in df.columns:
            print(f"Column '{col}' not found. Skipping.")
            continue

        current_type = df[col].dtype

        # If already correct dtype, skip
        if pd.api.types.is_dtype_equal(current_type, desired_type):
            continue

        try:
            if desired_type == "datetime":
                df[col] = pd.to_datetime(df[col], errors="coerce")
            else:
                df[col] = df[col].astype(desired_type)
            print(f"Converted column '{col}' from {current_type} to {desired_type}.")
        except Exception as e:
            print(f"Failed to convert column '{col}' from {current_type} to {desired_type}: {e}")

    return df

In [185]:
# Define the expected dtypes
dtype_mapping = {
    "reported_datetime": "datetime",
    "closed_datetime": "datetime",
    "id": "str",
    "status": "str",
    "latitude": "float",
    "longitude": "float",
    "median_income": "float",
    "total_population": "int",
    "average_age": "float"
}

# Apply the function
df_merged = enforce_column_dtypes(df_merged, dtype_mapping)

Converted column 'reported_datetime' from object to datetime.
Converted column 'closed_datetime' from object to datetime.
Converted column 'id' from object to str.
Converted column 'status' from object to str.
Converted column 'total_population' from float64 to int.


# 1.4 **Data Engineering**

### 1.4.1 **Retrieving Holiday Dates**

In [186]:
# U.S. holidays per relevant state
us_holidays = {
    "chicago": holidays.US(state="IL"),  # Illinois
    "sanfrancisco": holidays.US(state="CA"),  # California
    "newyork": holidays.US(state="NY")  # New York
}

# Add public holiday flag
def make_holiday_checker(date_col, city_col):
    def check_us_holiday(row):
        date = row[date_col].date()
        city = row[city_col]
        return date in us_holidays.get(city, [])
    return check_us_holiday

holiday_checker = make_holiday_checker("reported_datetime", "city")
df_merged["is_public_holiday"] = df_merged.apply(holiday_checker, axis=1)

### 1.4.2 **Standardise Issue Types into SG Categories**

**Normalise Issue Types**

In [220]:
def normalize_issue(issue):
    issue = issue.lower().strip()
    issue = re.sub(r'[\-\/]', ' ', issue)  # Replace dashes/slashes with spaces
    issue = re.sub(r'\s+', ' ', issue)  # Collapse multiple spaces
    return issue

**Keyword Mapping Setup**

In [229]:
# Original mapping dictionary
issue_type_mapping = {
    "Illegal Parking": [
        "illegal parking",
        "blocked driveway",
        "parking enforcement",
        "parking violation",
        "parking offence",
        "vehicle obstruction",
        "roadside parking"
    ],
    "Facilities in HDB Estates": [
        "street light pole damage",
        "damaged street sign",
        "sign repair request",
        "sign repair request - stop sign",
        "sign repair request - one way sign",
        "sign repair request - do not enter sign",
        "noise - residential",
        "noise - house of worship",
        "noise - commercial",
        "noise - aircraft",
        "elevator",
        "boilers",
        "paint/plaster",
        "flooring/stairs",
        "general request",
        "electrical",
        "plumbing"
    ],
    "Roads & Footprints": [
        "street condition",
        "blocked street",
        "sidewalk cleaning",
        "blocked sidewalk",
        "alley sewer",
        "traffic signal",
        "street and sidewalk cleaning",
        "traffic",
        "obstruction",
        "noise - vehicle"
    ],
    "Cleanliness": [
        "dirty condition",
        "litter",
        "litter basket complaint",
        "litter receptacle maintenance",
        "street cleaning",
        "graffiti",
        "graffiti public",
        "graffiti private",
        "illegal dumping",
        "residential disposal",
        "commercial disposal",
        "sanitation code violation",
        "posting advertisement",
        "illegal postings",
        "unsanitary condition",
        "missed collection",
        "panhandling",
        "residential disposal complaint",
        "commercial disposal complaint",
        "posting advertisement",
        "missed collection",
        "sanitation code violation"
    ],
    "Pests": [
        "rodent",
        "rat",
        "vicious animal",
        "animal-abuse"
    ],
    "Animals & Bird": [
        "animal complaint",
        "dead animal",
        "animal in a park",
        "report an injured animal",
        "stray dog",
        "wildlife",
        "violation of park rules"
    ],
    "Smoking": [
        "illegal smoking",
        "smoking offence",
        "urinating in public",
        "drinking",
        "drug activity"
    ],
    "Parks & Greenery": [
        "tree emergency",
        "tree debris",
        "fallen tree",
        "park maintenance",
        "animal in a park",
        "violation of park rules",
        "noise - park"
    ],
    "Drains & Sewers": [
        "sewer",
        "clogged drain",
        "flood",
        "water on street",
        "water in basement",
        "check for leak",
        "sewer cleaning",
        "alley sewer inspection"
    ],
    "Drinking Water": [
        "check for leak",
        "water system",
        "water quality",
        "hydrant issue"
    ],
    "Construction Sites": [
        "construction noise",
        "general construction",
        "building/use",
        "illegal construction",
        "real time enforcement",
        "special projects inspection team",
        "general construction/plumbing"
    ],
    "Abandoned Trolleys": [
        "abandoned cart",
        "abandoned vehicle",
        "abandoned vehicle complaint",
        "derelict vehicles"
    ],
    "Shared Bicycles": [
        "bike blocking path",
        "divvy bike parking complaint",
        "e-scooter parking complaint",
        "bike/roller/skate chronic"
    ],
    "Others": [
        "311 information only call",
        "emergency response team",
        "encampment",
        "homeless person assistance",
        "vendor enforcement",
        "institution disposal complaint",
        "muni employee feedback",
        "disorderly youth",
        "electrical",
        "emergency response team (ert)",
        "special projects inspection team (spit)",
        "general",
        "real time enforcement"
    ]
}


**Flashtext Processor Setup**

In [230]:
# Set up flashtext for fast keyword lookup
keyword_processor = KeywordProcessor()
for category, keywords in issue_type_mapping.items():
    for keyword in keywords:
        keyword_processor.add_keyword(keyword.lower(), category)

**Fuzzy Matching**

In [231]:
def fuzzy_match_category(issue, threshold=80):
    issue = str(issue).lower()
    best_match = ("Others", 0)
    for category, keywords in issue_type_mapping.items():
        for keyword in keywords:
            score = fuzz.partial_ratio(issue, keyword)
            if score > best_match[1]:
                best_match = (category, score)
    return best_match[0] if best_match[1] >= threshold else "Others"


**GPT-4 LLM Categorisation**

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # $env:OPENAI_API_KEY = "api-key-here"

# Store GPT results to avoid repeated API calls
gpt_cache = {}

def gpt_fallback_category(issue):
    issue = str(issue).strip().lower()
    
    if issue in gpt_cache:
        return gpt_cache[issue]
    
    prompt = f"""
    Categorise the following municipal issue into one of these Singapore categories:
    {list(issue_type_mapping.keys())}.
    Issue: "{issue}"
    Category:
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        result = response.choices[0].message.content.strip()
        # Remove any surrounding quotes or labels like "Category: 'Cleanliness'"
        result = re.sub(r'["“”‘’\']', '', result).strip()
        result = result.replace("Category:", "").strip()
        gpt_cache[issue] = result
        return result
    except Exception as e:
        print(f"OpenAI API Error: {e}")
        return "Others"

**Hybrid Issue Matching Setup & Execution**   

In [232]:
def clean_category_label(label):
    label = label.strip()
    label = re.sub(r"^[\"'‘’“”]+|[\"'‘’“”]+$", "", label)  # Strip leading/trailing quotes
    return label

def map_to_sg_category_hybrid(issue):
    issue = normalize_issue(issue)
    
    # FlashText fast match
    matches = keyword_processor.extract_keywords(issue)
    if matches:
        return matches[0]

    # Fuzzy fallback
    fuzzy_result = fuzzy_match_category(issue)
    if fuzzy_result != "Others":
        return fuzzy_result

    # GPT fallback
    return gpt_fallback_category(issue)

In [233]:
# Load cache from file if it exists
cache_path = "cache/gpt_cache.json"
if os.path.exists(cache_path):
    with open(cache_path, "r") as f:
        gpt_cache = json.load(f)
else:
    gpt_cache = {}

# Save cache to file after processing
def save_cache():
    with open(cache_path, "w") as f:
        json.dump(gpt_cache, f, indent=2)

In [234]:
df_merged["issue_type_sg"] = df_merged["issue_type"].apply(map_to_sg_category_hybrid).apply(clean_category_label)
save_cache()

In [240]:
# Quick check
print("Mapped SG Issue Types:")
print(df_merged["issue_type_sg"].value_counts())

Mapped SG Issue Types:
issue_type_sg
Illegal Parking              2757
Others                       2535
Facilities in HDB Estates    2142
Roads & Footprints           1966
Cleanliness                   227
Abandoned Trolleys            134
Smoking                        93
Construction Sites             72
Parks & Greenery               64
Pests                          41
Drains & Sewers                40
Drinking Water                 25
Shared Bicycles                12
Animals & Bird                  7
Name: count, dtype: int64


# 1.5 **Final Check & Saving**

In [ ]:
# Save to intermediate file for EDA
df_merged.to_csv("../data/files/cleaned_reports.csv", index=False)